# Hand reverse engineering slack emoji completion algorithm

In [79]:
import pathlib
import pandas as pd
import re
import Levenshtein
import json
from tqdm.auto import tqdm

In [1]:
import sys
sys.path.append('..')
import emojicompletion

/Users/keisuke.ogaki/.ghq/github.com/Hi-king/learn_slack_emoji_completion/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
candidates, case_dict = emojicompletion.data.SlackEmojiCompletionDataset(
    directory=pathlib.Path('..') /
    'data').load(filter_by_vocabulary=True)


In [77]:
params = json.load(open('../results/2input_l2_lstm_batch50_lr0.001_commitea975c7_hardnegFalse_20220706103635/params.json'))
keys_test = params['keys_test']

In [8]:
def distance(query, target):
    if not set(query).issubset(target):
        return 99999
    else:
        return target.find(query[0])


In [63]:
re.findall('a.*?a.*?a', 'bakaabbaa')

['akaa']

In [65]:
'.*?'.join('aaa')

'a.*?a.*?a'

In [ ]:
def distance(query, target):

    if not set(query).issubset(target):
        return 99999
    else:
        return target.find(query[0])

In [74]:
def distance2(query, target):

    if not set(query).issubset(target):
        return 9999999999
    else:
        pattern = '.*?'.join(query)
        matches = re.findall(pattern, target)
        if not matches:
            return 999999999
        else:
            return min(len(found) for found in matches)*100+target.find(query[0])

In [83]:
def distance3(query, target):

    if not set(query).issubset(target):
        return 9999999999
    else:
        pattern = '.*?'.join(query)
        matches = re.findall(pattern, target)
        if not matches:
            return 999999999
        else:
            return min(len(found) for found in matches)*100+len(target)

In [29]:
def get_rank(key, target_true, distance_func):
    distances = [distance_func(key, target) for target in candidates]
    df = pd.DataFrame(dict(
        key=[key] * len(candidates),
        distance=distances,
        target=list(candidates),
    ))
    df = df.sort_values(by="distance", ascending=True).assign(rank=range(1, len(df)+1))
    return df[df["target"] == target_true].iloc[0]['rank']

In [24]:
get_rank('aaa', 'kaaba')

237

In [25]:
keys = ['it', 'dg', 'hy', 'get', 'vlo']

In [31]:
for key in keys:
    print(key, get_rank(key, case_dict[key][0], distance_func=distance))

it 1
dg 24
hy 20
get 93
vlo 7


In [75]:
for key in keys:
    print(key, get_rank(key, case_dict[key][0], distance_func=distance2))

it 1
dg 1
hy 1
get 5
vlo 1


In [37]:
for key in keys:
    print(key, get_rank(key, case_dict[key][0], distance_func=Levenshtein.distance))

it 1
dg 64
hy 148
get 1566
vlo 344


In [38]:
for key in keys:
    print(key, get_rank(key, case_dict[key][0], distance_func=Levenshtein.jaro_winkler))

it 1891
dg 84
hy 1028
get 320
vlo 1323


In [78]:
def get_rank_df(key, distance_func):
    distances = [distance_func(key, target) for target in candidates]
    df = pd.DataFrame(dict(
        key=[key] * len(candidates),
        distance=distances,
        target=list(candidates),
    ))
    df = df.sort_values(by="distance", ascending=True).assign(rank=range(1, len(df)+1))
    return df

In [80]:
all_df = pd.DataFrame(index=candidates)
for key in tqdm(keys_test[:100]):
    df = get_rank_df(key, distance_func=distance2)
    all_df = all_df.assign(**{key: df.set_index("target")["rank"]})
    

100%|██████████| 100/100 [00:01<00:00, 69.58it/s]


In [82]:
n = 0
ranks = 0
for key in all_df.columns:
    targets = case_dict[key]
    n += len(targets)
    ranks += all_df.loc[targets, key].sum()
print(n, ranks, ranks/n)

82 1452 17.70731707317073


In [84]:
all_df = pd.DataFrame(index=candidates)
for key in tqdm(keys_test[:100]):
    df = get_rank_df(key, distance_func=distance3)
    all_df = all_df.assign(**{key: df.set_index("target")["rank"]})
    

100%|██████████| 100/100 [00:02<00:00, 42.71it/s]


In [85]:
n = 0
ranks = 0
for key in all_df.columns:
    targets = case_dict[key]
    n += len(targets)
    ranks += all_df.loc[targets, key].sum()
print(n, ranks, ranks/n)

82 1227 14.963414634146341
